In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.


<table align="left">
  <td><a href="https://colab.research.google.com/github/GoogleCloudPlatform/ai-ml-recipes/blob/main/notebooks/analytics/identifying_customer_segments.ipynb"><img src="https://avatars.githubusercontent.com/u/33467679?s=200&v=4" width="32px" alt="Colab logo"> Run in Colab</a></td>
  <td><a href="https://github.com/GoogleCloudPlatform/ai-ml-recipes/blob/main/notebooks/analytics/identifying_customer_segments.ipynb"><img src="https://github.githubassets.com/assets/GitHub-Mark-ea2971cee799.png" width="32px" alt="GitHub logo"> View on GitHub</a></td>
  <td><a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/ai-ml-recipes/main/notebooks/analytics/identifying_customer_segments.ipynb"><img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"> Open in Vertex AI Workbench</a></td>
  <td><a href="https://console.cloud.google.com/bigquery/import?url=https://github.com/GoogleCloudPlatform/ai-ml-recipes/blob/main/notebooks/analytics/identifying_customer_segments.ipynb"><img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTW1gvOovVlbZAIZylUtf5Iu8-693qS1w5NJw&s" alt="BQ logo" width="35"> Open in BQ Studio</a></td>
  <td><a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fai-ml-recipes%2Fmain%2Fnotebooks/analytics/identifying_customer_segments.ipynb"><img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"> Open in Colab Enterprise</a></td>
</table>

# Identifying Customer Segments for Targeted Marketing

## Overview

**The Business Challenge:** Marketing teams often need to optimize budget allocation to maximize return on investment (ROI). A one-size-fits-all marketing strategy is inefficient, leading to wasted ad spend and low engagement, as the messaging is not relevant to all recipients. The business problem is to develop a systematic, data-driven method for partitioning a customer base into distinct groups based on their behavior, enabling more effective and personalized marketing campaigns.

While basic segmentation using simple demographic data is straightforward, it often fails to capture the more nuanced differences in customer purchasing patterns. The analytical challenge is to move beyond these simple heuristics and identify meaningful segments based on complex behavioral data, such as purchase frequency, monetary value, and product category preferences. The goal is to produce segments that are not only statistically distinct but also interpretable and actionable for the marketing team.


**The Data Science Approach:** In this use case, we will combine unsupervised machine learning with generative AI to create and characterize customer segments. First, we will apply a k-means clustering algorithm directly within BigQuery ML to efficiently partition the entire customer dataset based on purchasing behavior within BigQuery.

Clustering effectively groups customers and generates a cluster ID. The second part of our approach is to automate the interpretation of these segments to provide business context. We will use a generative AI function to analyze the behavioral data of customers within each cluster and programmatically generate qualitative descriptions, including concise segment name, segment summary and tailored marketing suggestions for each segment.

## Setup

This section sets up the necessary environment and configurations for running the notebook. It includes defining project and dataset IDs and providing instructions for BigQuery connections.

Installs the `bigquery-magics` extension, which allows running BigQuery SQL queries directly within Jupyter notebooks.

In [ ]:
%pip install bigquery-magics -q

Loads the `bigquery_magics` extension, enabling the use of `%%bigquery --project $PROJECT_ID` cell magic for executing SQL.

In [ ]:
%load_ext bigquery_magics

### Configure Google Cloud Project and Dataset

Defines the Google Cloud project ID, dataset ID, and geographic location to be used for BigQuery operations. **Remember to replace `PROJECT_ID` and `DATASET_ID` with your actual values.**

In [ ]:
PROJECT_ID = "PROJECT_ID"
DATASET_ID = "DATASET_ID"
LOCATION = "US"

## Data Preparation

### Preview Raw Customer Order Data

This cell queries a sample of the raw customer order data from the `thelook_ecommerce` public dataset to understand its structure and content before further processing.

In [ ]:
%%bigquery --project $PROJECT_ID --project $PROJECT_ID
SELECT * FROM `bigquery-public-data.thelook_ecommerce.orders` LIMIT 1000;

Query is running:   0%|          |

Downloading:   0%|          |

,order_id,user_id,status,gender,created_at,returned_at,shipped_at,delivered_at,num_of_item
0,15,10,Cancelled,F,2025-07-19 06:06:00+00:00,NaT,NaT,NaT,1
1,18,14,Cancelled,F,2024-07-28 03:35:00+00:00,NaT,NaT,NaT,3
2,39,32,Cancelled,F,2021-05-19 16:28:00+00:00,NaT,NaT,NaT,1
3,47,37,Cancelled,F,2023-03-02 01:10:00+00:00,NaT,NaT,NaT,1
4,68,54,Cancelled,F,2023-11-28 10:53:00+00:00,NaT,NaT,NaT,1
...,...,...,...,...,...,...,...,...,...
995,12816,10141,Cancelled,F,2024-10-01 11:40:00+00:00,NaT,NaT,NaT,1
996,12833,10152,Cancelled,F,2025-09-03 01:47:50.414919+00:00,NaT,NaT,NaT,1
997,12869,10183,Cancelled,F,2025-06-29 15:24:00+00:00,NaT,NaT,NaT,1
998,12875,10185,Cancelled,F,2024-06-17 17:06:00+00:00,NaT,NaT,NaT,1


### Create Customer Features Table

This code aggregates raw order data to create a `customer_date` table, which includes `total_spend`, `number_of_orders`, and `last_purchase_date` for each customer. This table will be used for customer segmentation.

### Create BigQuery Dataset

Defines a BigQuery SQL query to create a new dataset if it doesn't already exist. This dataset will store all tables created during this analysis.

In [ ]:
#@title Create BigQuery dataset if not exists

query = f"""
CREATE SCHEMA IF NOT EXISTS `{PROJECT_ID}.{DATASET_ID}`
  OPTIONS (
    location = '{LOCATION}');
"""

Executes the SQL query to create the BigQuery dataset.

In [ ]:
%%bigquery --project $PROJECT_ID
$query

### Define Customer Features Table Creation Query

Defines a BigQuery SQL query to create or replace the `customer_date` table. This table aggregates customer information such as `total_spend`, `number_of_orders`, and `last_purchase_date` for each customer from the public e-commerce dataset.

In [ ]:
query = f"""
CREATE OR REPLACE TABLE {PROJECT_ID}.{DATASET_ID}.customer_date AS(
  SELECT
  users.id,
  SUM(order_items.sale_price) AS total_spend,
  COUNT(DISTINCT orders.order_id) AS number_of_orders,
  MAX(orders.created_at) AS last_purchase_date
FROM
  `bigquery-public-data.thelook_ecommerce.users` AS users
JOIN
  `bigquery-public-data.thelook_ecommerce.orders` AS orders
ON
  users.id = orders.user_id
JOIN
  `bigquery-public-data.thelook_ecommerce.order_items` AS order_items
ON
  orders.order_id = order_items.order_id
GROUP BY
  users.id );
"""

Executes the SQL query to create the `customer_date` table.

In [ ]:
%%bigquery --project $PROJECT_ID
$query

### Preview Prepared Customer Data

Displays a sample of the newly created `customer_date` table, confirming the data preparation step was successful and the features are correctly structured.

### Define Query to Preview Prepared Data

Defines a BigQuery SQL query to select a sample of data from the `customer_date` table.

In [ ]:
query = f"""
SELECT * FROM `{PROJECT_ID}.{DATASET_ID}.customer_date` LIMIT 1000;
"""

Executes the SQL query to display the first 1000 rows of the `customer_date` table.

In [ ]:
%%bigquery --project $PROJECT_ID
$query

## Customer Segmentation with BigQuery ML

### Train a K-Means Clustering Model

Utilizes BigQuery ML to train a k-means clustering model. This model will group customers into distinct segments based on their `total_spend`, `number_of_orders`, and `last_purchase_date`.

### Define K-Means Model Training Query

Defines a BigQuery ML SQL query to create and train a K-Means clustering model. The model is configured to create 3 clusters and standardize features for better performance.

In [ ]:
query = f"""
CREATE OR REPLACE MODEL `{PROJECT_ID}.{DATASET_ID}.customer_clusters`
OPTIONS(model_type='kmeans', num_clusters=3, standardize_features=TRUE) AS
SELECT
  id,
  total_spend,
  number_of_orders,
  last_purchase_date
FROM `{PROJECT_ID}.{DATASET_ID}.customer_date`;
"""

Executes the SQL query to train the K-Means clustering model.

In [ ]:
%%bigquery --project $PROJECT_ID
$query

### Assign Customers to Segments

Applies the trained k-means model using `ML.PREDICT` to assign each customer in the `customer_date` table to a specific cluster, creating a `predictions` table with their assigned `centroid_id`.

### Define Query to Assign Customers to Clusters

Defines a BigQuery SQL query to apply the trained K-Means model using `ML.PREDICT` to assign each customer in the `customer_date` table to a specific cluster, creating a `predictions` table with their assigned `centroid_id`.

In [ ]:
query = f"""
CREATE OR REPLACE TABLE `{PROJECT_ID}.{DATASET_ID}.predictions` AS(
SELECT
id,
centroid_id,
total_spend,
number_of_orders,
last_purchase_date
FROM
  ML.PREDICT(MODEL `{PROJECT_ID}.{DATASET_ID}.customer_clusters`,
    (SELECT * FROM
       `{PROJECT_ID}.{DATASET_ID}.customer_date`)

));
"""

Executes the SQL query to generate and save customer cluster predictions.

In [ ]:
%%bigquery --project $PROJECT_ID
$query

### Review Customer Segment Assignments

Displays a sample of the `predictions` table, showing customers along with their assigned cluster IDs and relevant features.

### Define Query to Review Segment Assignments

Defines a BigQuery SQL query to select a sample of data from the `predictions` table.

In [ ]:
query = f"""
SELECT * FROM `{PROJECT_ID}.{DATASET_ID}.predictions` LIMIT 50;
"""

Executes the SQL query to display the first 50 rows of the `predictions` table, showing assigned customer segments.

In [ ]:
%%bigquery --project $PROJECT_ID
$query

## Generate Segment Personas with AI

Use the AI.GENERATE_TABLE() function to analyze the members of each cluster and automatically generate a descriptive persona, a summary of their behavior, and potential marketing strategies for that segment.


### Create BigQuery ML Remote Model for AI Functions

Establishes a BigQuery ML remote model that connects to a generative AI endpoint (Gemini-2.5-Flash). This model enables the use of AI functions like `AI.GENERATE_TABLE` within BigQuery for advanced data analysis.

### Define Remote Model for Generative AI

Defines a BigQuery ML SQL query to create a remote model for integrating with generative AI (Gemini-2.5-Flash). This model will be used to generate textual descriptions for customer segments.

In [ ]:
# Create a BQML model to use the AI functions

query = f"""
CREATE OR REPLACE MODEL`{PROJECT_ID}.{DATASET_ID}.ai_model`
REMOTE WITH CONNECTION DEFAULT
OPTIONS (ENDPOINT = 'gemini-2.5-flash');
"""

Executes the SQL query to create the BigQuery ML remote model, connecting it to the specified generative AI endpoint.

In [ ]:
%%bigquery --project $PROJECT_ID
$query

### Generate Segment Profiles and Marketing Strategies

Leverages the `AI.GENERATE_TABLE` function to analyze the characteristics of each customer segment (cluster) and generate a concise segment name, a summary of their behavior, and tailored marketing suggestions.

### Define Query to Generate Segment Profiles and Marketing Strategies

Defines a BigQuery SQL query that uses the `AI.GENERATE_TABLE` function to analyze each customer segment's aggregated metrics (average spend, orders, and recency). It then prompts the generative AI model to create a segment name, summary, and marketing suggestions for each cluster. The output schema is explicitly defined for structured results.

In [ ]:
query = f"""
CREATE OR REPLACE TABLE `{PROJECT_ID}.{DATASET_ID}.segment_campaigns` AS(
SELECT
  *
FROM
  AI.GENERATE_TABLE(
    MODEL `{PROJECT_ID}.{DATASET_ID}.ai_model`,
    (
      SELECT
          'For the customer segment with centroid_id, provide a concise segment_name, a segment_summary (1-2 sentences), and 3 numbered marketing_suggestions in a list based on the provided metrics. For context, these metrics are from an ecommerce store.'
          AS prompt
      FROM (
        SELECT
          CAST(centroid_id AS STRING) AS centroid_id,
          AVG(total_spend) AS avg_total_spend,
          AVG(number_of_orders) AS avg_number_of_orders,
          AVG(TIMESTAMP_DIFF(CURRENT_TIMESTAMP(), last_purchase_date, DAY)) AS avg_time_since_last_purchase_days
        FROM
          `{PROJECT_ID}.{DATASET_ID}.predictions`
        GROUP BY
          centroid_id
      )
    ),
     STRUCT ("customer_segment STRING, segment_name STRING, segment_summary STRING, marketing_suggestions ARRAY<STRING>" AS output_schema ))
);
"""

Executes the SQL query to generate the customer segment profiles and marketing strategies using generative AI and stores them in the `segment_campaigns` table.

In [ ]:
%%bigquery --project $PROJECT_ID
$query

### Review Generated Segment Campaigns

Displays the AI-generated segment personas, including their names, summaries, and proposed marketing strategies, providing actionable insights for targeted marketing.

### Define Query to Review Generated Segment Campaigns

Defines a BigQuery SQL query to retrieve all generated segment campaigns.

In [ ]:
query = f"""
SELECT * FROM `{PROJECT_ID}.{DATASET_ID}.segment_campaigns`
"""

Executes the SQL query to display the final AI-generated customer segment personas and their corresponding marketing campaign suggestions.

In [ ]:
%%bigquery --project $PROJECT_ID
$query